In [92]:
# Dependencies 
import pandas as pd
import matplotlib as plt
from pathlib import Path
import numpy as np


In [67]:
# File to Load
file = Path("../Resources/U.S._Chronic_Disease_Indicators.csv")


In [68]:
# Read U.S chronic desease data File and store into Pandas DataFrame
chronic_df = pd.read_csv(file, encoding="ISO-8859-1")

# Data Frame chronic disease, 5 first rows 
chronic_df.head()


,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,TopicID,QuestionID,ResponseID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2019,2019,GA,Georgia,BRFSS,Disability,Adults with any disability,NaN,%,Crude Prevalence,...,DIS,DIS01,NaN,CRDPREV,AGE,AGE65P,NaN,NaN,NaN,NaN
1,2019,2019,GU,Guam,BRFSS,Arthritis,Arthritis among adults,NaN,%,Crude Prevalence,...,ART,ART01,NaN,CRDPREV,SEX,SEXF,NaN,NaN,NaN,NaN
2,2019,2019,GU,Guam,BRFSS,Immunization,Influenza vaccination among adults,NaN,%,Crude Prevalence,...,IMM,IMM01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
3,2019,2019,ME,Maine,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
4,2019,2019,NV,Nevada,NVSS,Health Status,Life expectancy at birth,NaN,Years,Number,...,HEA,HEA06,NaN,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN


#Cleaning data

In [69]:
# Exploring to see all our columns 
chronic_df.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'Response', 'DataValueUnit', 'DataValueType',
       'DataValue', 'DataValueAlt', 'DataValueFootnoteSymbol',
       'DataValueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'StratificationCategory3', 'Stratification3',
       'Geolocation', 'LocationID', 'TopicID', 'QuestionID', 'ResponseID',
       'DataValueTypeID', 'StratificationCategoryID1', 'StratificationID1',
       'StratificationCategoryID2', 'StratificationID2',
       'StratificationCategoryID3', 'StratificationID3'],
      dtype='object')

In [70]:
# Exploring the whole data 
chronic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311745 entries, 0 to 311744
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   YearStart                  311745 non-null  int64  
 1   YearEnd                    311745 non-null  int64  
 2   LocationAbbr               311745 non-null  object 
 3   LocationDesc               311745 non-null  object 
 4   DataSource                 311745 non-null  object 
 5   Topic                      311745 non-null  object 
 6   Question                   311745 non-null  object 
 7   Response                   0 non-null       float64
 8   DataValueUnit              311745 non-null  object 
 9   DataValueType              311745 non-null  object 
 10  DataValue                  210684 non-null  float64
 11  DataValueAlt               210684 non-null  float64
 12  DataValueFootnoteSymbol    102758 non-null  object 
 13  DataValueFootnote          10

In [71]:
# Create a boolean series indicating columns with zero non-null values
empty_column_filter_series = chronic_df.count() == 0 
# Display the boolean series
empty_column_filter_series

YearStart                    False
YearEnd                      False
LocationAbbr                 False
LocationDesc                 False
DataSource                   False
Topic                        False
Question                     False
Response                      True
DataValueUnit                False
DataValueType                False
DataValue                    False
DataValueAlt                 False
DataValueFootnoteSymbol      False
DataValueFootnote            False
LowConfidenceLimit           False
HighConfidenceLimit          False
StratificationCategory1      False
Stratification1              False
StratificationCategory2       True
Stratification2               True
StratificationCategory3       True
Stratification3               True
Geolocation                  False
LocationID                   False
TopicID                      False
QuestionID                   False
ResponseID                    True
DataValueTypeID              False
StratificationCatego

In [102]:
# Selects the count of non-null values for columns identified as empty by empty_column_filter_series
chronic_df.count()[empty_column_filter_series]

Response                     0
StratificationCategory2      0
Stratification2              0
StratificationCategory3      0
Stratification3              0
ResponseID                   0
StratificationCategoryID2    0
StratificationID2            0
StratificationCategoryID3    0
StratificationID3            0
dtype: int64

In [73]:
# Get the names of columns with no data (identified by empty_column_filter_series)
empty_columns = chronic_df.count()[empty_column_filter_series].index
# Display empty_columns variable 
empty_columns

Index(['Response', 'StratificationCategory2', 'Stratification2',
       'StratificationCategory3', 'Stratification3', 'ResponseID',
       'StratificationCategoryID2', 'StratificationID2',
       'StratificationCategoryID3', 'StratificationID3'],
      dtype='object')

In [65]:
# Remove columns with no data (identified by empty_columns) from the DataFrame chronic_df
# inplace=True ensures the changes are made to chronic_df directly
chronic_df.drop(columns=empty_columns, inplace=True)

In [35]:
# Print the shape of the DataFrame chronic_df (number of rows, number of columns)
print(chronic_df.shape)
# Print the column names of the DataFrame chronic_df after dropping non-null columns 
chronic_df.columns

(311745, 24)


Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'DataValueUnit', 'DataValueType', 'DataValue',
       'DataValueAlt', 'DataValueFootnoteSymbol', 'DataValueFootnote',
       'LowConfidenceLimit', 'HighConfidenceLimit', 'StratificationCategory1',
       'Stratification1', 'Geolocation', 'LocationID', 'TopicID', 'QuestionID',
       'DataValueTypeID', 'StratificationCategoryID1', 'StratificationID1'],
      dtype='object')

#How has the prevalence of chronic diseases changed from 2015 to 2022, and are there any significant trends or patterns over time?

In [38]:
# Count occurrences of each year in the "YearStart" column
# We want to see this result to understand the distribution of data over the years
chronic_df["YearStart"].value_counts()

YearStart
2019    88823
2021    87600
2020    67999
2022    47125
2016     8760
2015     8760
2018     2678
Name: count, dtype: int64

In [103]:
# Count instances of each location abbreviation in the "LocationAbbr" column
chronic_df["LocationAbbr"].value_counts()

LocationAbbr
US    5809
TN    5796
GA    5794
ND    5794
MA    5794
OR    5794
MI    5794
MO    5794
CO    5794
NY    5794
CT    5794
AR    5794
MS    5794
VA    5794
WI    5794
DE    5794
NE    5794
HI    5794
UT    5794
LA    5794
MN    5794
IL    5794
DC    5794
WA    5794
VT    5794
ME    5794
PA    5794
WY    5794
NJ    5794
KS    5794
AL    5794
MT    5788
SD    5786
NM    5786
NH    5745
MD    5745
FL    5745
IA    5745
KY    5745
WV    5745
AK    5737
NC    5696
RI    5696
OK    5696
AZ    5688
CA    5647
OH    5647
IN    5647
TX    5647
ID    5647
NV    5647
SC    5647
PR    4212
VI    4015
GU    4015
Name: count, dtype: int64

In [105]:
# Count instances of each location description grouped by location abbreviation
location_counts = chronic_df.groupby("LocationAbbr")["LocationDesc"].value_counts()
location_counts


LocationAbbr  LocationDesc        
AK            Alaska                  5737
AL            Alabama                 5794
AR            Arkansas                5794
AZ            Arizona                 5688
CA            California              5647
CO            Colorado                5794
CT            Connecticut             5794
DC            District of Columbia    5794
DE            Delaware                5794
FL            Florida                 5745
GA            Georgia                 5794
GU            Guam                    4015
HI            Hawaii                  5794
IA            Iowa                    5745
ID            Idaho                   5647
IL            Illinois                5794
IN            Indiana                 5647
KS            Kansas                  5794
KY            Kentucky                5745
LA            Louisiana               5794
MA            Massachusetts           5794
MD            Maryland                5745
ME            Maine

In [40]:
# Count the number of unique values in each column of chronic_df
chronic_df.nunique()

YearStart                        7
YearEnd                          4
LocationAbbr                    55
LocationDesc                    55
DataSource                      14
Topic                           19
Question                       109
DataValueUnit                    8
DataValueType                   12
DataValue                    14852
DataValueAlt                 14852
DataValueFootnoteSymbol          9
DataValueFootnote                9
LowConfidenceLimit            8412
HighConfidenceLimit           8861
StratificationCategory1          5
Stratification1                 26
Geolocation                     54
LocationID                      55
TopicID                         19
QuestionID                     109
DataValueTypeID                 12
StratificationCategoryID1        5
StratificationID1               26
dtype: int64

In [47]:
# Sorting our locationAbbr provide a better view to analyse this column 
sorted(chronic_df["LocationAbbr"].unique())

['AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'GU',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'PR',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'US',
 'UT',
 'VA',
 'VI',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY']

In [106]:
# Create a boolean series indicating rows where the "LocationAbbr" column is "US"
us_filter_series = chronic_df["LocationAbbr"] == "US"

# Filter the DataFrame chronic_df using the boolean series us_filter_series
# to get only rows where the "LocationAbbr" column is "US"
us_df = chronic_df[us_filter_series]

# Display the resulting DataFrame containing only rows where the location abbreviation is "US"
us_df

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,TopicID,QuestionID,ResponseID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
18,2020,2020,US,United States,BRFSS,Asthma,Current asthma among adults,NaN,%,Crude Prevalence,...,AST,AST02,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
19,2020,2020,US,United States,BRFSS,Oral Health,All teeth lost among adults aged 65 years and ...,NaN,%,Crude Prevalence,...,ORH,ORH03,NaN,CRDPREV,SEX,SEXF,NaN,NaN,NaN,NaN
20,2020,2020,US,United States,BRFSS,Oral Health,Visited dentist or dental clinic in the past y...,NaN,%,Crude Prevalence,...,ORH,ORH06,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
34,2016,2020,US,United States,US Cancer DVT,Cancer,"Invasive cancer (all sites combined), incidence",NaN,"per 100,000",Crude Rate,...,CAN,CAN07,NaN,CRDRATE,SEX,SEXF,NaN,NaN,NaN,NaN
39,2022,2022,US,United States,BRFSS,Health Status,2 or more chronic conditions among adults,NaN,%,Crude Prevalence,...,HEA,HEA05,NaN,CRDPREV,AGE,AGE4564,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311706,2022,2022,US,United States,BRFSS,Tobacco,Quit attempts in the past year among adult cur...,NaN,%,Crude Prevalence,...,TOB,TOB06,NaN,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
311708,2022,2022,US,United States,BRFSS,Sleep,Short sleep duration among adults,NaN,%,Crude Prevalence,...,SLEP,SLP03,NaN,CRDPREV,RACE,ASN,NaN,NaN,NaN,NaN
311714,2022,2022,US,United States,BRFSS,Social Determinants of Health,Lack of reliable transportation in the past 12...,NaN,%,Age-adjusted Prevalence,...,SDOH,SDH06,NaN,AGEADJPREV,RACE,ASN,NaN,NaN,NaN,NaN
311721,2022,2022,US,United States,BRFSS,"Nutrition, Physical Activity, and Weight Status",Obesity among adults,NaN,%,Crude Prevalence,...,NPAW,NPW14,NaN,CRDPREV,RACE,ASN,NaN,NaN,NaN,NaN


In [80]:
#What are the the most chronic deseases in the US
chronic_des= us_df["Topic"].unique()
chronic_des

array(['Asthma', 'Oral Health', 'Cancer', 'Health Status',
       'Nutrition, Physical Activity, and Weight Status', 'Sleep',
       'Disability', 'Alcohol', 'Tobacco', 'Immunization',
       'Cardiovascular Disease', 'Chronic Obstructive Pulmonary Disease',
       'Arthritis', 'Mental Health', 'Social Determinants of Health',
       'Cognitive Health and Caregiving', 'Diabetes',
       'Chronic Kidney Disease', 'Maternal Health'], dtype=object)

In [81]:
#numbe of chronic deseases
len(chronic_des)

19

In [96]:
#the count per chronic desease
years = us_df["YearStart"].unique()
years_sorted = sorted(years)
years_sorted


[2015, 2016, 2018, 2019, 2020, 2021, 2022]

In [ ]:
#